In [2]:
import requests
import json
#from typing import Tuple
import time
import datetime
import random
import pickle
from login import Login
import pandas as pd
import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/ 537.36 Edg/107.0.1418.62'
}

In [11]:
# 先不写了